### this script has been taken from the checkFiltering and DivideTrialsIntoReadingAndWriting script, and so all the filtering is based on the same

In [1]:
%run Functions.py

In [2]:
def GetBlinkPropertiesForEvents_onSections(EventTrial_index, PupilData_df, MissingData_startNlength):
    
    samplingFrequency = 90
    nSections = 4
    
    
    blinkFrequencyListSection = list()
    blinkDurationAverageListSection = list()
    blinkDurationTotalListSection = list()
    blinkCountListSection = list()
    interBlinkDurationListSection = list()
        
    
    # for every trial event, find the blinks during that event
    for ind, eventStart in enumerate(EventTrial_index['start']):
        eventEnd = EventTrial_index['end'][ind]
        
        blinkFrequencyList = list()
        blinkDurationAverageList = list()
        blinkDurationTotalList = list()
        blinkCountList = list()
        interBlinkDurationList = list()
        
        nSamples = eventEnd - eventStart + 1
        
        # blink properties for each of the 4 sections of the trial
        for nSection in range(nSections):
            eventStartSection = eventStart + nSection*int(nSamples/4)
            eventEndSection = eventStart + (nSection+1)*int(nSamples/4)
            
            blinkInd_missingData = np.array([indBlink for indBlink in range(0, len(MissingData_startNlength)) if \
                MissingData_startNlength[indBlink][0] > eventStartSection and MissingData_startNlength[indBlink][0] < \
                                         eventEndSection])
            blinkDurations_trial = np.array([MissingData_startNlength[indBlink][1] for indBlink in blinkInd_missingData])
            interBlinkDuration_trial = [MissingData_startNlength[indBlink][0] - MissingData_startNlength[indBlink-1][0] - \
                                  MissingData_startNlength[indBlink-1][1] for indBlink in blinkInd_missingData[1:]]
        
        
        
            trialTime = ((PupilData_df['timeStamp'][eventEndSection] - PupilData_df['timeStamp']\
                          [eventStartSection]).total_seconds())/60
            
            
            blinkCountList.append(len(blinkInd_missingData))
            blinkFrequencyList.append(len(blinkInd_missingData)/trialTime)
            
            blinkDurationTotalList.append(sum(blinkDurations_trial)/samplingFrequency)
            if len(interBlinkDuration_trial) > 0:
                interBlinkDurationList.append(np.mean(interBlinkDuration_trial)/samplingFrequency)
            else:
                interBlinkDurationList.append(0.0)
                
            if len(blinkDurations_trial) > 0:
                blinkDurationAverageList.append(np.mean(blinkDurations_trial)/samplingFrequency)
            else:
                blinkDurationAverageList.append(0.0)
            
            MissingData_startNlength_array = np.array(MissingData_startNlength)
            
        
        
        blinkCountListSection.append(blinkCountList)
        blinkFrequencyListSection.append(blinkFrequencyList)
        blinkDurationAverageListSection.append(blinkDurationAverageList)
        blinkDurationTotalListSection.append(blinkDurationTotalList)
        interBlinkDurationListSection.append(interBlinkDurationList)
        
        
        
        #print(MissingData_startNlength_array[blinkInd_missingData])
        #print(len(blinkInd_missingData)/trialTime)
        #print(sum(blinkDurations_trial)/samplingFrequency)
        #print(np.mean(blinkDurations_trial)/samplingFrequency)
        
    Blink_df = pd.DataFrame(list(zip(blinkCountListSection, blinkFrequencyListSection, blinkDurationAverageListSection, \
                                     blinkDurationTotalListSection,interBlinkDurationListSection)), columns=[\
                                    'blinkCount', 'blinkFrequency', 'blinkDurationAverage', 'blinkDurationTotal', \
                                    'interBlinkDuration'])
    Blink_df = Blink_df.fillna(0)
    
    
    return Blink_df

In [3]:
metricComputed_count = 'BlinkCount_sections'
metricComputed_freq = 'BlinkFrequency_sections'
metricComputed_durTotal = 'BlinkDurationTotal_sections'
metricComputed_durAvg = 'BlinkDurationAverage_sections'
metricComputed_interDur = 'InterBlinkDuration_sections'

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName_count = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_count +  '.xlsx'
resultFileName_freq = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_freq +  '.xlsx'
resultFileName_durTotal = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_durTotal +  '.xlsx'
resultFileName_durAvg = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_durAvg +  '.xlsx'
resultFileName_interDur = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Blinks\Subject_Block_Session_Trial_' + metricComputed_interDur +  '.xlsx'



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        #if 'ac\\4\\' not in root:
        #    continue
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            print(subjAndSessionName)
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            userKeys_new_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root, \
                                                   userKeys_new_wDwellTime)
            
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                blinkCount = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkDurationTotal = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkDurationAverage = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                blinkFrequency = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                interBlinkDuration = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
            
            else:
                # filter the data
                pupilData_filtered, interpolatedIndices, missingData_fromBlinks, missingData_overall, \
                missingDataBlink_startNlength, missingDataAll_startNlength  = FilterPupilSize_wBlinkData(gazeLog, \
                                                                            timeTyping, subjAndSessionName)
                
                """
                print('missing data ratio: ', sum(missingData_overall)/len(missingData_overall))
                print('interpolated data ratio: ', sum(interpolatedIndices)/len(interpolatedIndices))
                print('missing data from blinks ratio: ', sum(missingData_fromBlinks)/len(missingData_fromBlinks))
                
                blinkDurations = [item[1] for item in missingDataBlink_startNlength]
                fig=plt.figure(), plt.hist(blinkDurations)
                plt.title('blinkDurationHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\blinkDurationHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                missingDataAllDurations = [item[1] for item in missingDataAll_startNlength]
                fig=plt.figure(), plt.hist(missingDataAllDurations)
                plt.title('MissingDataLengthHistograms')
                pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Blinks\MissingDataLengthHistograms'+'\\'+subjName+'\\'+subjAndSessionName, 'wb'))
                
                
                """
                #print(missingDataBlink_startNlength)
                
                # divide trials into reading and writing
                eventReading, eventReading_index = EventPartsFromPupilData(eventTrialsInKeysSelected_reading, pupilData_filtered, root)
                eventWriting, eventWriting_index = EventPartsFromPupilData(eventTrialsInKeysSelected_writing, pupilData_filtered, root)
                
                eventTrial, eventTrial_index = CombineReadingWriting(eventReading, eventWriting, eventReading_index, eventWriting_index)
                
                #print('eventReadingIndex: ', eventReading_index)
                #print('')
                #print('eventTrialIndex:  ', eventTrial_index)
                
                # compute blink durations for all trials
                blink = GetBlinkPropertiesForEvents_onSections(eventTrial_index, pupilData_filtered, missingDataBlink_startNlength)
                
                
                blinkCount = blink['blinkCount'].tolist()
                blinkFrequency = blink['blinkFrequency'].tolist()
                blinkDurationAverage = blink['blinkDurationAverage'].tolist()
                blinkDurationTotal = blink['blinkDurationTotal'].tolist()
                interBlinkDuration = blink['interBlinkDuration'].tolist()
                
            if '1stPart' in root:
                print('1stPart')
                blinkCount1 = blinkCount
                blinkFrequency1 = blinkFrequency
                blinkDurationAverage1 = blinkDurationAverage
                blinkDurationTotal1 = blinkDurationTotal
                interBlinkDuration1 = interBlinkDuration
                
                continue
            
            if '2ndPart' in root:
                print('2ndPart')
                blinkCount2 = blinkCount
                blinkFrequency2 = blinkFrequency
                blinkDurationAverage2 = blinkDurationAverage
                blinkDurationTotal2 = blinkDurationTotal
                interBlinkDuration2 = interBlinkDuration
                
                blinkCount = blinkCount1 + blinkCount2
                blinkFrequency = blinkFrequency1 + blinkFrequency2
                blinkDurationAverage = blinkDurationAverage1 + blinkDurationAverage2
                blinkDurationTotal = blinkDurationTotal1 + blinkDurationTotal2
                interBlinkDuration = interBlinkDuration1 + interBlinkDuration2
                
                blinkCount1 = list()
                blinkFrequency1 = list()
                blinkDurationAverage1 = list()
                blinkDurationTotal1 = list()
                interBlinkDuration1 = list()
                
            
            # save the blink count
            dataToSave_count = DataForEveryTrial()
            dataToSave_count.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_count.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_count.sessionNumber = subjAndSessionName[-1]
            dataToSave_count.variable = metricComputed_count
            dataToSave_count.dataForTrial = blinkCount
            dataToSave_count.resultPathName = resultFileName_count
            
            #print(dataToSave_count.printInfo())
            
            
            # save the blink frequency
            dataToSave_freq = DataForEveryTrial()
            dataToSave_freq.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_freq.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_freq.sessionNumber = subjAndSessionName[-1]
            dataToSave_freq.variable = metricComputed_freq
            dataToSave_freq.dataForTrial = blinkFrequency
            dataToSave_freq.resultPathName = resultFileName_freq
            
            print(dataToSave_freq.printInfo())
            
            # save the blink duration total
            dataToSave_durTotal = DataForEveryTrial()
            dataToSave_durTotal.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_durTotal.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_durTotal.sessionNumber = subjAndSessionName[-1]
            dataToSave_durTotal.variable = metricComputed_durTotal
            dataToSave_durTotal.dataForTrial = blinkDurationTotal
            dataToSave_durTotal.resultPathName = resultFileName_durTotal
            
            #print(dataToSave_durTotal.printInfo())
            
            
            # save the blink duration average
            dataToSave_durAvg = DataForEveryTrial()
            dataToSave_durAvg.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_durAvg.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_durAvg.sessionNumber = subjAndSessionName[-1]
            dataToSave_durAvg.variable = metricComputed_durAvg
            dataToSave_durAvg.dataForTrial = blinkDurationAverage
            dataToSave_durAvg.resultPathName = resultFileName_durAvg
            
            print(dataToSave_durAvg.printInfo())
            
            # save the inter blink duration 
            dataToSave_interDur = DataForEveryTrial()
            dataToSave_interDur.subjectID = subjAndSessionName.split('__')[0]
            dataToSave_interDur.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave_interDur.sessionNumber = subjAndSessionName[-1]
            dataToSave_interDur.variable = metricComputed_interDur
            dataToSave_interDur.dataForTrial = interBlinkDuration
            dataToSave_interDur.resultPathName = resultFileName_interDur
            
            print(dataToSave_interDur.printInfo())
            
            
            
            #dataToSave_count.AddToFile()
            #dataToSave_freq.AddToFile()
            #dataToSave_durTotal.AddToFile()
            #dataToSave_durAvg.AddToFile()
            #dataToSave_interDur.AddToFile()
            
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
ac__1__2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
  subjectID block session  trial  \
0        ac     1       1      0   
1        ac     1       1      1   
2        ac     1       1      2   
3        ac     1       1      3   
4        ac     1       1      4   

                             BlinkFrequency_sections  
0  [36.49377167894767, 14.230555876684237, 16.276...  
1  [27.124571629551586, 13.560906438108136, 10.86...  
2  [10.817985766776125, 16.22804660839235, 10.818...  
3  [34.30502311300933, 10.147462244844599, 12.152...  
4  [15.453803515877668, 10.301426610233163, 10.30...  
  subjectID block session  trial  \
0        ac     1       1      0   
1        ac     1       1      1   
2        ac     1       1      2   
3        ac     1       1      3   
4        ac     1       1      4   

                       BlinkDurationAverage_sections  
0  [0.18580246913580245, 0.1603174603174603

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-03d94d7dcc3c>", line 142, in <module>
    userKeys_new_wDwellTime)
  File "C:\DTU\PythonScripts\Analysis_Expt_201901\Functions.py", line 735, in FindReadingPartsOfTrial_inKeysSelected
    userKeysTime = timeConversion(userKeysTimeStr)
  File "C:\DTU\PythonScripts\Analysis_Expt_201901\Functions.py", line 556, in timeConversion
    timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
  File "C:\ProgramData\Anaconda3\lib\_strptime.py", line 577, in _strptime_datetime
    tt, fraction, gmtoff_fraction = _strptime(data_string, format)
  File "C:\ProgramData\Anaconda3\lib\_strptime.py", line 378, in _strptime
    for group_key in found_dict.keys():
KeyboardInterrupt

During handling of the above exception, another exception oc

KeyboardInterrupt: 